In [15]:
import librosa
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import os
from tqdm import tqdm
import logging
import time
import math

# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


class FeatureExtractor:
    def __init__(self, sr=22050, frame_size=2048, hop_length=512):
        self.sr = sr
        self.frame_size = frame_size
        self.hop_length = hop_length
        self.spectrogram = None  # Cache spectrogram for reuse

    def compute_spectrogram(self, audio):
        """Compute spectrogram once and cache it."""
        if self.spectrogram is None:
            self.spectrogram = librosa.stft(
                audio, n_fft=self.frame_size, hop_length=self.hop_length
            )
        return self.spectrogram

    def extract_features(self, audio):
        features = {}

        # Time domain features
        features["zcr"] = np.mean(self.extract_zcr(audio))
        features["rms"] = np.mean(self.extract_rms(audio))
        features["amplitude_envelope"] = np.mean(self.extract_amplitude_envelope(audio))

        # Frequency domain features
        features["spectral_centroid"] = np.mean(self.extract_spectral_centroid(audio))
        features["spectral_bandwidth"] = np.mean(self.extract_spectral_bandwidth(audio))
        features["band_energy_ratio"] = np.mean(
            self.band_energy_ratio(audio, split_frequency=1000)
        )
        features["spectral_rolloff"] = np.mean(self.extract_spectral_rolloff(audio))
        # features["spectral_contrast"] = self.extract_spectral_contrast(
        #     audio
        # )  # 1D array of shape (7,)
        features["spectral_flatness"] = np.mean(self.extract_spectral_flatness(audio))
        features["mfcc"] = self.extract_mfcc(audio)  # 1D array of shape (13,)
        features["mel_spectrogram"] = np.mean(self.mel_spectrogram(audio))
        features["spectrogram"] = np.mean(self.extract_spectrogram(audio))

        # New features
        # features["chroma"] = self.extract_chroma(audio)  # 1D array of shape (12,)
        features["cqt"] = np.mean(self.extract_cqt(audio))
        features["pitch"] = np.mean(self.extract_pitch(audio))
        # features["tonnetz"] = self.extract_tonnetz(audio)  # 1D array of shape (6,)
        features["tempo"] = self.extract_tempo(audio)  # Scalar
        features["beat_interval"] = self.extract_beat_interval(audio)  # Scalar
        features["onset_interval"] = self.extract_onset_interval(audio)  # Scalar

        self.spectrogram = None  # Clear cached spectrogram
        return features

    def extract_zcr(self, audio):
        zcr = librosa.feature.zero_crossing_rate(
            audio, frame_length=self.frame_size, hop_length=self.hop_length
        )[0]
        return zcr if zcr.size > 0 else np.array([0.0])

    def extract_rms(self, audio):
        rms = librosa.feature.rms(
            y=audio, frame_length=self.frame_size, hop_length=self.hop_length
        )[0]
        return rms if rms.size > 0 else np.array([0.0])

    def extract_amplitude_envelope(self, audio):
        amplitude_envelope = np.array(
            [
                max(audio[i : i + self.frame_size])
                for i in range(0, len(audio), self.hop_length)
            ]
        )
        return amplitude_envelope if amplitude_envelope.size > 0 else np.array([0.0])

    def extract_spectral_centroid(self, audio):
        centroid = librosa.feature.spectral_centroid(
            y=audio, sr=self.sr, n_fft=self.frame_size, hop_length=self.hop_length
        )[0]
        return centroid if centroid.size > 0 else np.array([0.0])

    def extract_spectral_bandwidth(self, audio):
        bandwidth = librosa.feature.spectral_bandwidth(
            y=audio, sr=self.sr, n_fft=self.frame_size, hop_length=self.hop_length
        )[0]
        return bandwidth if bandwidth.size > 0 else np.array([0.0])

    def band_energy_ratio(self, audio, split_frequency=1000):
        spectrogram = self.compute_spectrogram(audio)

        num_frequency_bins = self.frame_size // 2 + 1
        frequency_range = self.sr / 2
        frequency_delta_per_bin = frequency_range / num_frequency_bins
        split_frequency_bin = int(math.floor(split_frequency / frequency_delta_per_bin))

        power_spectrogram = np.abs(spectrogram) ** 2
        power_spectrogram = power_spectrogram.T

        band_energy_ratio = []
        for frame in power_spectrogram:
            sum_power_low = frame[:split_frequency_bin].sum()
            sum_power_high = frame[split_frequency_bin:].sum()
            ber = sum_power_low / (sum_power_high + 1e-10)
            band_energy_ratio.append(ber)

        ber_array = np.array(band_energy_ratio)
        return ber_array if ber_array.size > 0 else np.array([0.0])

    def extract_spectral_rolloff(self, audio):
        rolloff = librosa.feature.spectral_rolloff(
            y=audio, sr=self.sr, n_fft=self.frame_size, hop_length=self.hop_length
        )[0]
        return rolloff if rolloff.size > 0 else np.array([0.0])

    def extract_spectral_contrast(self, audio):
        contrast = librosa.feature.spectral_contrast(
            y=audio, sr=self.sr, n_fft=self.frame_size, hop_length=self.hop_length
        )
        return np.mean(contrast, axis=0) if contrast.size > 0 else np.zeros(7)

    def extract_spectral_flatness(self, audio):
        flatness = librosa.feature.spectral_flatness(
            y=audio, n_fft=self.frame_size, hop_length=self.hop_length
        )[0]
        return flatness if flatness.size > 0 else np.array([0.0])

    def extract_spectrogram(self, audio):
        spectrogram = librosa.amplitude_to_db(
            self.compute_spectrogram(audio), ref=np.max
        )
        return (
            spectrogram
            if spectrogram.size > 0
            else np.zeros((self.frame_size // 2 + 1, 1))
        )

    def mel_spectrogram(self, audio):
        mel_spec = librosa.feature.melspectrogram(
            y=audio, sr=self.sr, hop_length=self.hop_length, n_mels=128
        )
        return mel_spec[0] if mel_spec.size > 0 else np.zeros(1)

    def extract_mfcc(self, audio, n_mfcc=13):
        mfccs = librosa.feature.mfcc(
            y=audio,
            sr=self.sr,
            n_fft=self.frame_size,
            hop_length=self.hop_length,
            n_mfcc=n_mfcc,
        )
        return np.mean(mfccs, axis=1) if mfccs.size > 0 else np.zeros(n_mfcc)

    def extract_chroma(self, audio):
        chroma = librosa.feature.chroma_stft(
            y=audio, sr=self.sr, n_fft=self.frame_size, hop_length=self.hop_length
        )
        return np.mean(chroma, axis=1) if chroma.size > 0 else np.zeros(12)

    def extract_cqt(self, audio):
        cqt = librosa.cqt(y=audio, sr=self.sr, hop_length=self.hop_length)
        return np.abs(cqt) if cqt.size > 0 else np.zeros((84, 1))

    def extract_pitch(self, audio):
        pitches, magnitudes = librosa.piptrack(y=audio, sr=self.sr)
        # Extract pitch values where magnitude is non-zero
        pitch_values = pitches[magnitudes > 0]
        return pitch_values if pitch_values.size > 0 else np.array([0.0])

    def extract_tonnetz(self, audio):
        tonnetz = librosa.feature.tonnetz(y=audio, sr=self.sr)
        return np.mean(tonnetz, axis=1) if tonnetz.size > 0 else np.zeros(6)

    def extract_tempo(self, audio):
        tempo = librosa.beat.tempo(y=audio, sr=self.sr)[0]
        return tempo if not np.isnan(tempo) else 0.0

    def extract_beat_interval(self, audio):
        tempo, beat_frames = librosa.beat.beat_track(y=audio, sr=self.sr)
        beat_times = librosa.frames_to_time(beat_frames, sr=self.sr)
        if len(beat_times) < 2:
            return 0.0
        intervals = np.diff(beat_times)
        return np.mean(intervals) if intervals.size > 0 else 0.0

    def extract_onset_interval(self, audio):
        onset_frames = librosa.onset.onset_detect(y=audio, sr=self.sr)
        onset_times = librosa.frames_to_time(onset_frames, sr=self.sr)
        if len(onset_times) < 2:
            return 0.0
        intervals = np.diff(onset_times)
        return np.mean(intervals) if intervals.size > 0 else 0.0


def extract_features(file_path, columns):
    try:
        start_time = time.time()
        audio, sr = librosa.load(file_path, sr=None, mono=True, duration=5.0)
        logging.info(f"Loaded {file_path} in {time.time() - start_time:.2f} seconds")

        if len(audio) < sr:
            logging.warning(
                f"Skipping {file_path}: Audio too short ({len(audio)/sr:.2f} seconds)"
            )
            return None

        start_time = time.time()
        extractor = FeatureExtractor(sr=sr)
        features = extractor.extract_features(audio)
        logging.info(
            f"Extracted features from {file_path} in {time.time() - start_time:.2f} seconds"
        )

        # Check for NaN values in features
        for key, val in features.items():
            if isinstance(val, (np.ndarray, float, int)):
                if isinstance(val, np.ndarray):
                    if np.isnan(val).any():
                        logging.warning(
                            f"Skipping {file_path}: Features contain NaN values in {key}"
                        )
                        return None
                else:
                    if np.isnan(val):
                        logging.warning(
                            f"Skipping {file_path}: Features contain NaN values in {key}"
                        )
                        return None

        # Flatten array-like features (mfcc, spectral_contrast, chroma, tonnetz) into separate columns
        flattened_features = {}
        for key, val in features.items():
            if isinstance(val, np.ndarray) and val.size > 1:  # For array-like features
                for i, v in enumerate(val):
                    flattened_features[f"{key}_{i}"] = v
            else:
                flattened_features[key] = val

        return flattened_features

    except Exception as e:
        logging.error(f"Failed to process {file_path}: {e}")
        return None


def process_directory(directory_path, label, batch_size=100, n_jobs=4):
    audio_files = [
        os.path.join(directory_path, f)
        for f in os.listdir(directory_path)
        if f.endswith((".wav", ".mp3"))
    ]
    logging.info(f"Found {len(audio_files)} audio files in {directory_path}")

    sample_file = audio_files[0] if audio_files else None
    if not sample_file:
        logging.warning(f"No audio files found in {directory_path}")
        return

    start_time = time.time()
    sample_features = extract_features(sample_file, columns=None)
    if sample_features is None:
        logging.warning(f"Could not determine columns from {sample_file}")
        return
    columns = list(sample_features.keys())
    logging.info(
        f"Determined {len(columns)} columns in {time.time() - start_time:.2f} seconds"
    )

    for i in range(0, len(audio_files), batch_size):
        batch_files = audio_files[i : i + batch_size]
        logging.info(
            f"Processing batch {i // batch_size + 1} with {len(batch_files)} files"
        )

        start_time = time.time()
        results = Parallel(n_jobs=n_jobs, verbose=10)(
            delayed(extract_features)(file_path, columns) for file_path in batch_files
        )
        logging.info(
            f"Processed batch {i // batch_size + 1} in {time.time() - start_time:.2f} seconds"
        )

        filtered_res = [item for item in results if item is not None]
        if not filtered_res:
            logging.warning(f"No valid results in batch {i // batch_size + 1}")
            continue

        for res in filtered_res:
            res["label"] = label

        start_time = time.time()
        df = pd.DataFrame(filtered_res, columns=columns + ["label"])
        logging.info(
            f"Created DataFrame with {len(df)} rows in {time.time() - start_time:.2f} seconds"
        )

        start_time = time.time()
        initial_rows = len(df)
        df = df.dropna()
        logging.info(
            f"Dropped {initial_rows - len(df)} rows with NaN values in {time.time() - start_time:.2f} seconds"
        )

        if df.empty:
            logging.warning(
                f"Batch {i // batch_size + 1} is empty after dropping NaN values"
            )
            continue

        start_time = time.time()
        output_file = f"{directory_path}/batch_{i // batch_size + 1}_features.csv"
        df.to_csv(output_file, index=False)
        logging.info(
            f"Saved batch {i // batch_size + 1} with {len(df)} entries to {output_file} in {time.time() - start_time:.2f} seconds"
        )

        del df, filtered_res, results


if __name__ == "__main__":
    base_dir = "processed"
    labels = {
        "Female_Fifties": 0,
        "Female_Twenties": 1,
        "Male_Fifties": 2,
        "Male_Twenties": 3,
    }
    batch_size = 100  # Keep small for initial timing
    n_jobs = -1

    for dir_name, label in labels.items():
        dir_path = os.path.join(base_dir, dir_name)
        if os.path.exists(dir_path):
            logging.info(f"Processing directory: {dir_path} with label {label}")
            process_directory(dir_path, label, batch_size, n_jobs)
        else:
            logging.warning(f"Directory {dir_path} does not exist")

2025-04-30 01:59:25,436 - INFO - Processing directory: processed\Female_Fifties with label 0
2025-04-30 01:59:25,489 - INFO - Found 15688 audio files in processed\Female_Fifties
2025-04-30 01:59:25,492 - INFO - Loaded processed\Female_Fifties\common_voice_en_10047.wav in 0.00 seconds
C:\Users\bedom\AppData\Local\Temp\ipykernel_15952\1145384127.py:140: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  spectrogram = librosa.amplitude_to_db(
C:\Users\bedom\AppData\Local\Temp\ipykernel_15952\1145384127.py:186: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  tempo = librosa.beat.tempo(y=audio, sr=self.sr)[0]
2025-04-30 01:59:25,766 - INFO - Extracted features from processed\Female_Fifties\common_voice_en_10047.wav in 0.27 seconds
2025-04-30 01:59:25,767